In [1]:
# montar google drive para guardar y acceder a los archivos del proyecto
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Entrar a la carpeta del proyecto en Google Drive y mostrar todos los archivos que contiene
%cd "/content/drive/MyDrive/Pedro Yanez - neutral-farming-om"
!ls -R


/content/drive/MyDrive/Pedro Yanez - neutral-farming-om
.:
cloudflared-linux-amd64.deb  model  README.md	      src
data			     om.db  requirements.txt

./data:
organic_matter_dataset.csv

./model:
artifacts

./model/artifacts:
metrics.json  model.joblib

./src:
api.py	__init__.py  __pycache__  train_model.py
db.py	models.py    schema.py	  utils.py

./src/__pycache__:
api.cpython-312.pyc  __init__.cpython-312.pyc  schema.cpython-312.pyc
db.cpython-312.pyc   models.cpython-312.pyc    utils.cpython-312.pyc


In [5]:
# instalar las dependencias necesarias del proyecto desde requirements.txt Va a pedir reiniciar y luego de eso hay que volver a ejecutar el paso anterior
# Es decir el paso de entrar a la carpeta de my drive
!pip install -r requirements.txt


In [6]:
# ejecutar el script de entrenamiento para crear el modelo y guardar métricas en model/artifacts
!python src/train_model.py


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
Saved: /content/drive/MyDrive/Pedro Yanez - neutral-farming-om/model/artifacts/model.joblib
Metrics: {
  "mae": 0.13336367648681424,
  "rmse": 0.14653228960860765,
  "r2": 0.9561801798001207,
  "n_train": 8,
  "n_val": 2
}


In [7]:
# abrir y mostrar las métricas de validación guardadas en model/artifacts/metrics.json
import json

with open("model/artifacts/metrics.json", "r") as f:
    metrics = json.load(f)

metrics


{'mae': 0.13336367648681424,
 'rmse': 0.14653228960860765,
 'r2': 0.9561801798001207,
 'n_train': 8,
 'n_val': 2}

In [8]:
# instalar pyngrok para exponer la API de FastAPI al exterior con un link público
!pip install pyngrok


In [9]:
# instalar cloudflared oficial. Luego detener con stop y ejecutar el siguiente comando, desde ahí acceder a la url para probar la api
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
!uvicorn src.api:app --host 0.0.0.0 --port 8000 & cloudflared tunnel --url http://localhost:8000


Selecting previously unselected package cloudflared.
(Reading database ... 126374 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.8.1) ...
Setting up cloudflared (2025.8.1) ...
Processing triggers for man-db (2.10.2-1) ...
2025-09-13T00:36:49Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-09-13T00:36:49Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-09-13T00:36:53Z I

In [10]:
# exponer la API en un link público sin password. Ir a una página del tipo https://deficit-thumbnail-refined-engine.trycloudflare.com/docs para probar la api
!uvicorn src.api:app --host 0.0.0.0 --port 8000 & cloudflared tunnel --url http://localhost:8000

2025-09-13T00:38:56Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-09-13T00:38:56Z INF Requesting new quick Tunnel on trycloudflare.com...
INFO:     Started server process [2796]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
2025-09-13T00:39:00Z INF +--------------------------------------------------------------------------------------------+
2025-0

In [ ]:
#Ir a post predict try out y e indicar
# Link de ejemplo: https://pmc-tracks-aged-locations.trycloudflare.com/docs

{
  "pH": 6.7,
  "EC": 0.25,
  "Total_Nitrogen": 0.12,
  "Moisture": 28
}


In [ ]:
# endpoint /predict para construir un DataFrame con los nombres EXACTOS de entrenamiento. NO ES NECESARIO ESTE PASO
from pathlib import Path

api_code = """
from __future__ import annotations
from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.orm import Session
from .db import Base, engine, SessionLocal
from .models import SoilRecord
from .schema import IngestPayload, PredictPayload, PredictResponse
from .utils import get_model

Base.metadata.create_all(bind=engine)
app = FastAPI(title="Neutral Farming OM API", version="0.1.0")

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

@app.post('/data_ingestion')
def data_ingestion(payload: IngestPayload, db: Session = Depends(get_db)):
    record = SoilRecord(
        pH=payload.pH,
        EC=payload.EC,
        total_nitrogen=payload.Total_Nitrogen,
        moisture=payload.Moisture,
        organic_matter=payload.Organic_Matter,
    )
    db.add(record)
    db.commit()
    db.refresh(record)
    return {'id': record.id}

@app.post('/predict', response_model=PredictResponse)
def predict(payload: PredictPayload):
    import pandas as pd
    model = get_model()
    # construir DataFrame con los mismos nombres de columnas usados al entrenar
    X = pd.DataFrame([{
        'pH': payload.pH,
        'EC': payload.EC,
        'Total Nitrogen': payload.Total_Nitrogen,  # OJO: espacio como en el CSV
        'Moisture': payload.Moisture
    }])
    try:
        pred = float(model.predict(X)[0])
    except Exception as e:
        raise HTTPException(status_code=400, detail=f'Prediction failed: {e}')
    return PredictResponse(predicted_organic_matter=round(pred, 4))
"""
Path("src/api.py").write_text(api_code)
print("api.py actualizado 🎯")


api.py actualizado 🎯


In [ ]:
# Prueba de data ingestion
# Ir al link de ejemplo https://pmc-tracks-aged-locations.trycloudflare.com/docs
# Ir a POST /data_ingestion, try out, y usar el siguiente json de ejemplo

{
  "pH": 6.9,
  "EC": 0.28,
  "Total_Nitrogen": 0.14,
  "Moisture": 29,
  "Organic_Matter": 4.1
}


In [12]:
import sqlite3 # Detener la ejecución en segundo plano del servidor FastAPI con Cloudflare Tunnel, para poder ejecutar este comando
# Con este comando aparecen las tablas que existen en SQLite
# Conectar a la base
conn = sqlite3.connect("om.db")

# Listar todas las tablas
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print("Tablas en la base:", tables)


Tablas en la base: [('soil_records',)]


In [14]:
import pandas as pd

# Leer todas las filas de la tabla soil_records. Son los que fueron insertados a través de Ingestion.
df = pd.read_sql_query("SELECT * FROM soil_records;", conn)

# Mostrar el dataframe
df



,id,pH,EC,total_nitrogen,moisture,organic_matter
0,1,6.9,0.28,0.14,29.0,4.1
1,2,13.0,0.20,0.30,99.0,0.2
